# Imports

In [1]:
import pandas as pd
import os
import requests
import sqlite3
import logging
import zipfile

# Download

In [2]:
# Criando diretório para armazenar os dados brutos (caso ainda não exista)
os.makedirs("../data/raw", exist_ok=True)

# Configurando o logging para registrar as etapas
logging.basicConfig(
    filename="../data/etl_log.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("Projeto ETL iniciado")

In [15]:
# URL do dataset de Covid-19 do Google
url_base   = "https://storage.googleapis.com/covid19-open-data/v3/"

# Diretório para salvar os dados
diretorio_dados = "../data/raw"
os.makedirs(diretorio_dados, exist_ok=True)

# Lista de arquivos CSV que queremos baixar
arquivos_csv = [
    "index.csv",
    "demographics.csv",
    "economy.csv",
    "epidemiology.csv",
    "lawatlas-emergency-declarations.csv",
    "geography.csv",
    "health.csv",
    "hospitalizations.csv",
    "mobility.csv",
    "oxford-government-response.csv",
    "worldbank.csv",
    "vaccinations.csv",
    "Global_vaccination_search_insights.csv",
    "by-sex.csv",
    "by-age.csv",
    "weather.csv",
    "google-search-trends.csv",
    "facility-boundary-us-all.csv"
]

# Loop para baixar os arquivos
for arquivo in arquivos_csv:
    url = url_base + arquivo
    caminho_arquivo = os.path.join(diretorio_dados, arquivo)

    print(f"Baixando {arquivo}...")
    response = requests.get(url)
    
    # Salvar o conteúdo do arquivo
    with open(caminho_arquivo, "wb") as file:
        file.write(response.content)
    
    print(f"{arquivo} salvo em {caminho_arquivo}")

print("Todos os arquivos foram baixados com sucesso!")


Baixando index.csv...
index.csv salvo em ../data/raw\index.csv
Baixando demographics.csv...
demographics.csv salvo em ../data/raw\demographics.csv
Baixando economy.csv...
economy.csv salvo em ../data/raw\economy.csv
Baixando epidemiology.csv...
epidemiology.csv salvo em ../data/raw\epidemiology.csv
Baixando lawatlas-emergency-declarations.csv...
lawatlas-emergency-declarations.csv salvo em ../data/raw\lawatlas-emergency-declarations.csv
Baixando geography.csv...
geography.csv salvo em ../data/raw\geography.csv
Baixando health.csv...
health.csv salvo em ../data/raw\health.csv
Baixando hospitalizations.csv...
hospitalizations.csv salvo em ../data/raw\hospitalizations.csv
Baixando mobility.csv...
mobility.csv salvo em ../data/raw\mobility.csv
Baixando oxford-government-response.csv...
oxford-government-response.csv salvo em ../data/raw\oxford-government-response.csv
Baixando worldbank.csv...
worldbank.csv salvo em ../data/raw\worldbank.csv
Baixando vaccinations.csv...
vaccinations.csv sal

In [2]:
# Diretório onde os arquivos foram salvos
diretorio_dados = "../data"

# Escolher um dos arquivos para testar (exemplo: epidemiology.csv)
arquivo_teste = "mobility.csv"
raw_data = os.path.join(diretorio_dados, 'raw')
caminho_arquivo = os.path.join(raw_data, arquivo_teste)

# Ler o CSV no Pandas
df = pd.read_csv(caminho_arquivo)

# Mostrar as primeiras linhas do dataframe
df.head()

,date,location_key,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential
0,2020-02-15,AE,0.0,4.0,5.0,0.0,2.0,1.0
1,2020-02-16,AE,1.0,4.0,4.0,1.0,2.0,1.0
2,2020-02-17,AE,-1.0,1.0,5.0,1.0,2.0,1.0
3,2020-02-18,AE,-2.0,1.0,5.0,0.0,2.0,1.0
4,2020-02-19,AE,-2.0,0.0,4.0,-1.0,2.0,1.0


# Salvando Banco de Dados

In [8]:
# Diretórios dos arquivos
diretorio_csv = "../data/raw"
caminho_db = "../covid_data.db"

# Criar conexão com o banco
conexao = sqlite3.connect(caminho_db)

# Listar todos os arquivos CSV na pasta
arquivos_csv = [f for f in os.listdir(diretorio_csv) if f.endswith(".csv")]

# Tamanho dos chunks
chunksize = 1000000

for arquivo in arquivos_csv:
    caminho_arquivo = os.path.join(diretorio_csv, arquivo)
    nome_tabela = arquivo.replace(".csv", "")  # Nome da tabela = nome do arquivo
    nome_tabela = nome_tabela.replace("-", "_")
    
    print(f"Salvando {arquivo} na tabela {nome_tabela}...")
    
    for chunk in pd.read_csv(caminho_arquivo, chunksize=chunksize):
        chunk.to_sql(nome_tabela, conexao, if_exists="append", index=False)
    
    print(f"{arquivo} salvo no banco de dados!")

# Fechar conexão
conexao.close()

print("Todos os arquivos foram inseridos no banco SQLite com sucesso!")


Salvando by-age.csv na tabela by_age...


C:\Users\Pedro\AppData\Local\Temp\ipykernel_16576\575194805.py:21: DtypeWarning: Columns (148,149,150,151) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(caminho_arquivo, chunksize=chunksize):
C:\Users\Pedro\AppData\Local\Temp\ipykernel_16576\575194805.py:21: DtypeWarning: Columns (151) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(caminho_arquivo, chunksize=chunksize):


by-age.csv salvo no banco de dados!
Salvando by-sex.csv na tabela by_sex...
by-sex.csv salvo no banco de dados!
Salvando demographics.csv na tabela demographics...
demographics.csv salvo no banco de dados!
Salvando economy.csv na tabela economy...
economy.csv salvo no banco de dados!
Salvando epidemiology.csv na tabela epidemiology...
epidemiology.csv salvo no banco de dados!
Salvando facility-boundary-us-all.csv na tabela facility_boundary_us_all...
facility-boundary-us-all.csv salvo no banco de dados!
Salvando geography.csv na tabela geography...
geography.csv salvo no banco de dados!
Salvando Global_vaccination_search_insights.csv na tabela Global_vaccination_search_insights...
Global_vaccination_search_insights.csv salvo no banco de dados!
Salvando google-search-trends.csv na tabela google_search_trends...
google-search-trends.csv salvo no banco de dados!
Salvando health.csv na tabela health...
health.csv salvo no banco de dados!
Salvando hospitalizations.csv na tabela hospitalizat

In [4]:
## Nome do banco de dados
banco_dados = "../covid_data.db"

# Conecta-se ao banco de dados SQlite
conexao = sqlite3.connect(banco_dados)

# Cria um cursor
cursor = conexao.cursor()

# Executa a consulta SQL para obter o nome das tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Recupera os resultados da consulta
tabelas = cursor.fetchall()

# Exibe o nome das tabelas  
print("Tabelas do banco de dados")
for tabela in tabelas:
    print(tabela[0])

## Fecha o cursor e a conexao com o banco
cursor.close()
conexao.close()

Tabelas do banco de dados
by_age
by_sex
demographics
economy
epidemiology
facility_boundary_us_all
geography
Global_vaccination_search_insights
google_search_trends
health
hospitalizations
index
lawatlas_emergency_declarations
mobility
oxford_government_response
vaccinations
weather
worldbank
